In [2]:
import gzip 
import json
from tqdm import tqdm
from unidecode import unidecode
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
PATH = "C:/Users/alpye/OneDrive/Desktop/DATA/coalition_3.gz"

In [ ]:
cityList = []

with open("data/cities.txt", "r", encoding = "utf-8") as file:
    for line in file:
        line = unidecode(line)
        line = line.strip().lower()
        cityList.append(line)

In [ ]:
# Now get them in a dictionary format, set all their values to 0 and save them as a JSON file

data_dict = {key: 0 for key in cityList}

file_path = 'data/main_cities.json'
with open(file_path, 'w') as file:
    json.dump(data_dict, file, indent=4)

In [ ]:
# This JSON file has populations of the cities in Turkey

with open("data/populations.json", "r") as file:
    populations = json.load(file)


In [ ]:
df = pd.read_excel('data/city_street.xlsx')

ilce_dict = {}
semt_dict = {}
mah_dict = {}

for i, row in df.iterrows():
    main_key = unidecode(row['il'].strip().lower())
    sub_key = unidecode(row['ilçe'].strip().lower())
    sub_sub_key = unidecode(row['semt_bucak_belde'].strip().lower())
    value = (unidecode(row["Mahalle"].strip().lower()))[:-4] # Exlude MAH

    ilce_dict.setdefault(main_key, {}).setdefault(sub_key, 0)
    semt_dict.setdefault(main_key, {}).setdefault(sub_sub_key, 0)
    mah_dict.setdefault(main_key, {}).setdefault(value, 0)

# DISCLAIMER: The values of these dictionaries were useless.
# I realized that while I was making progress...

In [ ]:
# Make 3 seperate lists to check in the future

ilce_list = []
semt_list = []
mah_list = []


for i, row in df.iterrows():
    ilce = unidecode(row['ilçe'].strip().lower())
    semt = unidecode(row['semt_bucak_belde'].strip().lower())
    mah = (unidecode(row["Mahalle"].strip().lower()))[:-4] # Exlude the MAH

    if ilce not in ilce_list:
        ilce_list.append(ilce)
    if semt not in semt_list:
        semt_list.append(semt)
    if mah not in mah_list:
        mah_list.append(mah)

In [ ]:
def find_main_key(nested_dict, middle_key):
    for main_key, inner_dict in nested_dict.items():
        if middle_key in inner_dict:
            return main_key
    return None

In [ ]:
# Get the JSON file as variable "data"

with open("data/main_cities.json", "r") as file:
    city_data = json.load(file)

# checks for ["user"]["location"], appends the user's id's in a list. Also adds to the city_data dictionary

user_list = []

memberCNT = 0
cnt = 0
with gzip.open(PATH, "rt") as fh:
    pbar = tqdm(fh)
    for i in pbar:
        uid, data = i.split("\t") 
        data = json.loads(data) # Makes the json into a Python dictionary
        for tweet in data:
            if tweet["user"]["location"] != '':
                loc = unidecode(tweet["user"]["location"].strip())
                loc = loc.lower()
                if "/" in loc:
                    loc = loc.split("/")
                elif "," in loc:
                    loc = loc.split(",")
                else:
                    loc = loc.split(" ")
                loc = [element.replace(" ", "") for element in loc]

                common_elements = set(loc).intersection(cityList)
                if common_elements:
                    city = list(common_elements)[0]
                    city_data[city] += 1
                    memberCNT += 1
                    user_list.append(tweet["user"]["id"])
                    break
                
                else:
                    common_elements = set(loc).intersection(ilce_list)
                    if common_elements:
                        if len(list(common_elements)) > 1:
                            final = 0
                            final_idx = -1
                            for i in list(common_elements):
                                search = find_main_key(ilce_dict, i)
                                idx = cityList.index(search)
                                pop = populations[idx]["population"]
                                if final < pop:
                                    final = pop
                                    final_idx = idx
                            
                            city = cityList[final_idx]
                        else:
                            ilce = list(common_elements)[0]
                            city = find_main_key(ilce_dict, ilce)
                        
                        city_data[city] += 1
                        memberCNT += 1
                        user_list.append(tweet["user"]["id"])
                        break
                    
                    else:
                        common_elements = set(loc).intersection(semt_list)
                        if common_elements:
                            if len(list(common_elements)) > 1:
                                final = 0
                                final_idx = -1
                                for i in list(common_elements):
                                    search = find_main_key(semt_dict, i)
                                    idx = cityList.index(search)
                                    pop = populations[idx]["population"]
                                    if final < pop:
                                        final = pop
                                        final_idx = idx
                            
                                city = cityList[final_idx]
                            else:
                                semt = list(common_elements)[0]
                                city = find_main_key(semt_dict, semt)
                            
                            city_data[city] += 1
                            memberCNT += 1
                            user_list.append(tweet["user"]["id"])
                            break

                        else:
                            common_elements = set(loc).intersection(mah_list)
                            if common_elements:
                                if len(list(common_elements)) > 1:
                                    final = 0
                                    final_idx = -1
                                    for i in list(common_elements):
                                        search = find_main_key(mah_dict, i)
                                        idx = cityList.index(search)
                                        pop = populations[idx]["population"]
                                        if final < pop:
                                            final = pop
                                            final_idx = idx
                                
                                    city = cityList[final_idx]
                                else:
                                    mah = list(common_elements)[0]
                                    city = find_main_key(mah_dict, mah)
                                
                                city_data[city] += 1
                                memberCNT += 1
                                user_list.append(tweet["user"]["id"])
                                break
            
            else:
                break
        cnt += 1
        pbar.set_postfix({"Count": cnt, "Successful Count": memberCNT})         

In [ ]:
# Runtime: 1 hour 21 minutes

In [ ]:
'''
Total users: 515 098
Users with legit location in their bio: 117 866
22.9%
'''

In [ ]:
file_path = 'data/main_cities_finished.json'
with open(file_path, 'w') as file:
    json.dump(city_data, file, indent=4)

with open('data/user_list.txt', 'w') as file:
    for item in user_list:
        file.write(str(item) + '\n')

In [ ]:
with open('data/user_list.txt', 'r') as file:
    user_list = [int(line.strip()) for line in file]

In [ ]:
with open("data/main_cities_finished.json", "r") as file:
    city_data = json.load(file)

In [ ]:
# checks for ["place"]["full_name"], appends the user's id's in a list. Also adds to the city_data dictionary

user_place_id = []

user_avoid = 0
memberCNT = 0
cnt = 0
with gzip.open(PATH, "rt") as fh:
    pbar = tqdm(fh)
    for i in pbar:
        uid, data = i.split("\t") 
        data = json.loads(data) # Makes the json into a Python dictionary
        user_dict = {key: 0 for key in cityList}
        went_in = False
        for tweet in data:
            if tweet["user"]["id"] == user_list[user_avoid]:
                if user_avoid < 117865:
                    user_avoid += 1
                break
            else:
                if tweet["place"] is not None:
                    loc = unidecode(tweet["place"]["full_name"].strip())
                    loc = loc.lower()
                    if "/" in loc:
                        loc = loc.split("/")
                    elif "," in loc:
                        loc = loc.split(",")
                    else:
                        loc = loc.split(" ")
                    loc = [element.replace(" ", "") for element in loc]

                    common_elements = set(loc).intersection(cityList)
                    if common_elements:
                        city = list(common_elements)[0]
                        user_dict[city] += 1
                        went_in = True
                        to_be_appended = tweet["user"]["id"]
                        
                    else:
                        common_elements = set(loc).intersection(ilce_list)
                        if common_elements:
                            if len(list(common_elements)) > 1:
                                final = 0
                                final_idx = -1
                                for i in list(common_elements):
                                    search = find_main_key(ilce_dict, i)
                                    idx = cityList.index(search)
                                    pop = populations[idx]["population"]
                                    if final < pop:
                                        final = pop
                                        final_idx = idx
                                
                                city = cityList[final_idx]
                            else:
                                ilce = list(common_elements)[0]
                                city = find_main_key(ilce_dict, ilce)
                            
                            user_dict[city] += 1
                            went_in = True
                            to_be_appended = tweet["user"]["id"]
                        
                        else:
                            common_elements = set(loc).intersection(semt_list)
                            if common_elements:
                                if len(list(common_elements)) > 1:
                                    final = 0
                                    final_idx = -1
                                    for i in list(common_elements):
                                        search = find_main_key(semt_dict, i)
                                        idx = cityList.index(search)
                                        pop = populations[idx]["population"]
                                        if final < pop:
                                            final = pop
                                            final_idx = idx
                                
                                    city = cityList[final_idx]
                                else:
                                    semt = list(common_elements)[0]
                                    city = find_main_key(semt_dict, semt)
                                
                                user_dict[city] += 1
                                went_in = True
                                to_be_appended = tweet["user"]["id"]
                                

                            else:
                                common_elements = set(loc).intersection(mah_list)
                                if common_elements:
                                    if len(list(common_elements)) > 1:
                                        final = 0
                                        final_idx = -1
                                        for i in list(common_elements):
                                            search = find_main_key(mah_dict, i)
                                            idx = cityList.index(search)
                                            pop = populations[idx]["population"]
                                            if final < pop:
                                                final = pop
                                                final_idx = idx
                                    
                                        city = cityList[final_idx]
                                    else:
                                        mah = list(common_elements)[0]
                                        city = find_main_key(mah_dict, mah)
                                    
                                    user_dict[city] += 1
                                    went_in = True
                                    to_be_appended = tweet["user"]["id"]


        if went_in:           
            city = max(user_dict, key = user_dict.get)
            city_data[city] += 1
            memberCNT += 1
            user_place_id.append(to_be_appended)
            went_in = False                            
                                
        cnt += 1
        pbar.set_postfix({"Count": cnt, "Successful Count": memberCNT, "User List idx": user_avoid})         

In [ ]:
'''
Runtime: 33:30
Users with legit geo info in their tweets (excluding the ones with geo info in their bio): 29 194
5.67%
'''

In [ ]:
file_path = 'data/main_cities_finished_2.json'
with open(file_path, 'w') as file:
    json.dump(city_data, file, indent=4)

In [ ]:
with open("data/main_cities_finished_2.json", "r") as file:
    city_data = json.load(file)

In [ ]:
with open('data/user_list_place.txt', 'w') as file:
    for item in user_place_id:
        file.write(str(item) + '\n')

In [ ]:
with open('data/user_list_place.txt', 'r') as file:
    user_place_id = [int(line.strip()) for line in file]

In [ ]:
# This ran on a script since it takes a long time, checked for ["geo"]

user_geo_id = []

geoCnt = 0
place_idx = 0
userloc_idx = 0
cnt = 0
with gzip.open(PATH, "rt") as fh:
    pbar = tqdm(fh)
    for i in pbar:
        uid, data = i.split("\t") 
        data = json.loads(data) # Makes the json into a Python dictionary
        user_dict = {key: 0 for key in cityList}
        went_in = False
        for tweet in data:
            if user_list[userloc_idx] == tweet["user"]["id"]:
                if userloc_idx < 117865:
                    userloc_idx += 1
                break
            if user_place_id[place_idx] == tweet["user"]["id"]:
                if place_idx < 29193:
                    place_idx += 1
                break
            if tweet["geo"] is not None:
                new_dict = {}
                new_dict["type"] = "Point"
                new_dict["coordinates"] = []
                new_dict["coordinates"].append(tweet["geo"]["coordinates"][1])
                new_dict["coordinates"].append(tweet["geo"]["coordinates"][0])
                with open("data/temp.geojson", "w") as f:
                    json.dump(new_dict, f)

                pt = gpd.read_file("data/temp.geojson")
                df = gpd.read_file("data/turkey.geojson")
                intersections = gpd.overlay(pt, df, how='intersection')
                name = intersections["name"]
                if name.empty is False:
                    city = name.iloc[0]
                    city = unidecode(city)
                    city = city.lower()
                    user_dict[city] += 1
                    to_be_appended = tweet["user"]["id"]
                    went_in = True

        if went_in:           
            city = max(user_dict, key = user_dict.get)
            city_data[city] += 1
            geoCnt += 1
            user_geo_id.append(to_be_appended)
            went_in = False
        cnt += 1
        pbar.set_postfix({"Count": cnt, "Successful Count": geoCnt, "UserBioLoc idx": userloc_idx, "TweetPlace idx": place_idx})

In [ ]:
'''
Runtime: 1 hour 45 minutes (on a script)
Users that had a ["geo"] but no user geo info nor tweet place info: 3893
'''

In [ ]:
with open('data/user_list_geo.txt', 'w') as file:
    for item in user_geo_id:
        file.write(str(item) + '\n')

In [ ]:
with open('data/user_list_geo.txt', 'r') as file:
    user_geo_id = [int(line.strip()) for line in file]

In [ ]:
file_path = 'data/main_cities_finished_3.json'
with open(file_path, 'w') as file:
    json.dump(city_data, file, indent=4)

In [ ]:
with open("data/main_cities_finished_3.json", "r") as file:
    city_data = json.load(file)

In [ ]:
'''
Total number of users: 515 098
Total number of users with the location info: 150 953
29.3%
'''

In [ ]:
# Got a dictionary to test
# Format: {user id: [tweet content]}

test_dict = {}

geo_idx = 0
place_idx = 0
userloc_idx = 0
cnt = 0
up = 0
with gzip.open(PATH, "rt", encoding="utf-8") as fh:
    pbar = tqdm(fh)
    for i in pbar:
        uid, data = i.split("\t") 
        data = json.loads(data) # Makes the json into a Python dictionary
        temp_list = []
        went_in_loc = False
        went_in_place = False
        went_in_geo = False
        for tweet in data:
            if user_list[userloc_idx] == tweet["user"]["id"]:
                temp_list.append(tweet["full_text"])
                went_in_loc = True
                key = user_list[userloc_idx] 

            elif user_place_id[place_idx] == tweet["user"]["id"]:
                temp_list.append(tweet["full_text"])
                went_in_place = True
                key = user_place_id[place_idx]

            elif tweet["user"]["id"] == user_geo_id[geo_idx]:
                temp_list.append(tweet["full_text"])
                went_in_geo = True
                key = user_geo_id[geo_idx]

        if went_in_loc:
            if userloc_idx < 117865:
                userloc_idx += 1
                test_dict[str(key)] = temp_list
                up += 1
                went_in_loc = False
        
        elif went_in_place:
            if place_idx < 29193:
                place_idx += 1
                test_dict[str(key)] = temp_list
                up += 1
                went_in_place = False

        elif went_in_geo:
            if geo_idx < 3892:
                geo_idx += 1
                test_dict[str(key)] = temp_list
                up += 1
                went_in_geo = False

        if up == 2:
            break
        cnt += 1
        pbar.set_postfix({"Count": cnt, "GeoIdx": geo_idx, "UserBioLoc idx": userloc_idx, "TweetPlace idx": place_idx, "DictElement Count": up})

# Runtime: 35 minutes
# 150950 keys (users)

In [ ]:
file_path = 'data/test_dict.json'
with open(file_path, 'w') as file:
    json.dump(test_dict, file)

In [3]:
with open("data/test_dict.json", "r") as file:
    test_dict = json.load(file)

In [4]:
for i in range(0,2):
    first_key = list(test_dict.keys())[i]
    first_value = test_dict[first_key]
    print(first_value)

['@MHP_Bilgi Senin son nefesin yakındır senin için sorun olmayacaktır. Peki bu vatanın gençliği geleceği ne olacaktır?', 'Herkes üniversite pişmanlıktır, üniversite okudum şimdi kuryeyim falan diyor. Madem kurye olcam şansımı yurt dışında denerim?', '@KadirMisiroglu @SebilYayinevi @OsmanlilarVakfi Leşini bu toprak kabul etcekmi sanıyosunuz kanı bozuk şerefsiz! Sen ileri geri konuşabilesin diye bu topraklarda şehitler verilmedi arap iti', 'Memleketimin araplaştığını görmektense sınır dışı edilip yeni bir hayat kurmayı yeğlerim! Araplara karşı sonuna kadar Irkçıyım!', '@aleynakrolu1 Yak aga yak yak', '@samiltayyar27 Ülkeyi boka sürükleyen isimler arasına girdi sayın Cumhurbaşkanı hala diyorsunuz ki reis. Üslübumu mazur görün fakat hakikat bu...', '@FOXhaber @fatihportakal #tavizverevere ülkenin uçuruma sürüklenmesine sebep olduk.Bir gazeteci olarak soruyorum...Şimdi ne olucak?', '@Alex10 Anlamıyoz ama seviyoz be KRAL', '@samiltayyar27 Bakıyo gene bugün nasıl Türkiyeye kalp krizi geçirtir